<a href="https://colab.research.google.com/github/CarlosYohei/ASSIGNMENT18/blob/main/Day07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DAY 07 - DNC
#Carlos Yohei - RID 4313
#Sistema de Recomendação

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Importação das tabelas

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

url = 'https://raw.githubusercontent.com/CarlosYohei/Day07/main/Data/DNC_orders_dataset.csv'
df_orders = pd.read_csv(url, index_col=0)
print("DNC_orders_dataset \n")
print("\n", df_orders.head(5))
print("\n", df_orders.info())
print("\n", df_orders.describe())
print(f"\n Registros duplicados", df_orders.duplicated().sum())

url = 'https://raw.githubusercontent.com/CarlosYohei/Day07/main/Data/DNC_order_items_dataset.csv'
df_items = pd.read_csv(url, index_col=0)
print("\n \n DNC_order_items_dataset \n")
print("\n", df_items.head(5))
print("\n", df_items.info())
print("\n", df_items.describe())
print(f"\n Registros duplicados", df_items.duplicated().sum())

url = 'https://raw.githubusercontent.com/CarlosYohei/Day07/main/Data/DNC_order_reviews_dataset.csv'
df_reviews = pd.read_csv(url, index_col=0)
print("\n \n DNC_order_reviews_dataset \n")
print("\n", df_reviews.head(5))
print("\n", df_reviews.info())
print("\n", df_reviews.describe())
print(f"\n Registros duplicados", df_reviews.duplicated().sum())

url = 'https://raw.githubusercontent.com/CarlosYohei/Day07/main/Data/DNC_products_dataset.csv'
df_products = pd.read_csv(url, index_col=0)
print("\n \n DNC_products_dataset \n")
print("\n", df_products.head(5))
print("\n", df_products.info())
print("\n", df_products.describe())
print(f"\n Registros duplicados", df_products.duplicated().sum())


DNC_orders_dataset 


                            order_id                       customer_id order_status
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d    delivered
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef    delivered
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089    delivered
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82    delivered
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c    delivered
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_id      99441 non-null  object
 1   customer_id   99441 non-null  object
 2   order_status  99441 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB

 None

                                 order_id                       customer_id order_status
count                   